In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import pandas as pd
import numpy as np
import librosa
from pathlib import Path
from tqdm import tqdm

/Users/muzammil/Desktop/ai_voice_coach_agent/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# ================= CONFIG =================
SAMPLE_RATE = 16000
MODEL_NAME = "facebook/wav2vec2-base"
PROCESSED_FEATURES_CSV = Path("data/processed/processed_features.csv")
BATCH_SIZE = 4
LEARNING_RATE = 1e-5
EPOCHS = 5  # 👈 you can change this anytime

# ================= DATASET =================
class EmotionAudioDataset(Dataset):
    def __init__(self, df, processor, label2id):
        self.df = df
        self.processor = processor
        self.label2id = label2id

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        y, _ = librosa.load(row["segment_path"], sr=SAMPLE_RATE)
        inputs = self.processor(y, sampling_rate=SAMPLE_RATE, return_tensors="pt", padding=True,return_attention_mask = True)
        label = self.label2id[row["emotion"]]
        return {
            "input_values": inputs.input_values.squeeze(0),
            "attention_mask": inputs.attention_mask.squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long),
            "dict": self.label2id
        }

# ================= MODEL =================

In [29]:
class Wav2Vec2EmotionClassifier(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.wav2vec = Wav2Vec2Model.from_pretrained(MODEL_NAME)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.wav2vec.config.hidden_size, num_labels)

    def forward(self, input_values, attention_mask=None):
        outputs = self.wav2vec(input_values, attention_mask=attention_mask)
        pooled = outputs.last_hidden_state.mean(dim=1)
        logits = self.classifier(self.dropout(pooled))
        return logits
    


In [30]:
## padding
def collate_fn(batch):
    """Pads variable-length audio inputs dynamically for Wav2Vec2."""
    input_values = [item["input_values"] for item in batch]
    labels = torch.stack([item["labels"] for item in batch])

    # Pad input_values to the same length
    input_values = torch.nn.utils.rnn.pad_sequence(
        input_values, batch_first=True
    )

    return {"input_values": input_values, "labels": labels}

# ================= TRAINING FUNCTIONS =================


In [31]:
def build_label_map(df):
    emotions = sorted(df["emotion"].unique())
    label2id = {e: i for i, e in enumerate(emotions)}
    id2label = {i: e for e, i in label2id.items()}
    return label2id, id2label

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for batch in tqdm(loader, desc="Training", leave=False):
        input_values = batch["input_values"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        logits = model(input_values)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(loader)


In [32]:
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for batch in tqdm(loader, desc="Validation", leave=False):
            input_values = batch["input_values"].to(device)
            labels = batch["labels"].to(device)
            logits = model(input_values)
            loss = criterion(logits, labels)
            total_loss += loss.item()

            preds = torch.argmax(logits, dim=-1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return total_loss / len(loader), correct / total


In [33]:
# ================= MAIN RUNNER =================

df = pd.read_csv(PROCESSED_FEATURES_CSV)
label2id, id2label = build_label_map(df)

processor = Wav2Vec2Processor.from_pretrained(MODEL_NAME)
dataset = EmotionAudioDataset(df, processor, label2id)

# simple train/val split
train_df = df.sample(frac=0.8, random_state=42)
val_df = df.drop(train_df.index)
train_ds = EmotionAudioDataset(train_df, processor, label2id)
val_ds = EmotionAudioDataset(val_df, processor, label2id)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = Wav2Vec2EmotionClassifier(num_labels=len(label2id)).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

print(f"🚀 Starting training for {EPOCHS} epochs on {device.upper()}")

# for epoch in range(EPOCHS):
#     train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
#     val_loss, val_acc = evaluate(model, val_loader, criterion, device)
#     print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.3f}")

# # Save model checkpoint
# model.wav2vec.save_pretrained("models/wav2vec2_emotion_finetuned")
# processor.save_pretrained("models/wav2vec2_emotion_finetuned")
# print("✅ Model saved as ")



/Users/muzammil/Desktop/ai_voice_coach_agent/.venv/lib/python3.11/site-packages/transformers/configuration_utils.py:335: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


🚀 Starting training for 5 epochs on CPU


In [36]:
with torch.no_grad():
    for batch in tqdm(train_loader, desc="Validation", leave=False):
        input_values = batch["input_values"].to(device)
        labels = batch["labels"].to(device)
        logits = model(input_values)
        print(batch["dict"].to(device))
        print(input_values)
        print(labels)
        print(logits)

KeyError: 'dict'

In [37]:
train_ds.label2id

{'angry': 0,
 'calm': 1,
 'disgust': 2,
 'fearful': 3,
 'happy': 4,
 'neutral': 5,
 'sad': 6,
 'surprised': 7}

In [38]:
MODEL_PATH = Path("./models/wav2vec2_emotion_finetuned")  # fine-tuned model dir
AUDIO_DIR = Path("data/processed/segments")
OUTPUT_CSV = Path("data/processed/emotion_predictions.csv")
BATCH_SIZE = 4

# ===== 1️⃣ Load model + processor =====
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Wav2Vec2ForSequenceClassification.from_pretrained(MODEL_PATH, local_files_only=True).to(device)
processor = Wav2Vec2Processor.from_pretrained(MODEL_PATH, local_files_only=True)
model.eval()

NameError: name 'Wav2Vec2ForSequenceClassification' is not defined

In [39]:
model

Wav2Vec2EmotionClassifier(
  (wav2vec): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
   